In [ ]:
import findspark
findspark.init()

In [ ]:
# from delta import *

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("testt").master("local").getOrCreate()

In [ ]:
data = [("abc","pqr"),("abc","pqr"),("gty",None)]
schema = ["name","value"]

df = spark.createDataFrame(data = data, schema = schema)
# df.show()

In [ ]:
df = df.schema.names
print(df)
df1 = ','.join(df)
# df4 = df1.map(lambda x:x,1)
print(df1)

In [ ]:
data = [("abc","pqr"),("abc","pqr")]
schema = ["name","value"]

df1 = spark.createDataFrame(data = data, schema = schema)
# df1.show()

In [ ]:

# %run logging.ipynb

In [ ]:
df2 = spark.read.format("csv").option("header","true").option("inferSchema", "true").option("skipFirst", "4").load("employees.csv")
# print(df2.count())

In [ ]:
df4 = spark.read.format("csv").option("header","true").option("inferSchema", "true").option("endingPosition", "-10").load("employees.csv")
print(df4.count())

In [ ]:
df_last = int(df2.count() * 0.5)
dff_last = df2.limit(df_last).show()

In [ ]:
df_half = df2.sample(fraction = 0.2)
print(df_half.count())

In [ ]:
df_hhalf = df2.limit(int(df2.count()/2))
print(df_hhalf.count())

In [ ]:
print(df2.schema.fields)

In [ ]:
from pyspark.sql.types import *
schema = [StructField(i.name ,StringType(),True) for i in df2.schema.fields]
new_schema = StructType(schema)
print(new_schema)

In [ ]:
print(list(set(df2.columns) - set(df.columns)))

In [ ]:
from pyspark.sql.functions import *
from functions import reduce
new_columns =["agv","hjk"]
df3 = reduce(lambda new_df, x : new_df.withColumn(x,lit(None)), new_colums, df2)
df3.show()

In [ ]:
from pyspark.sql.functions import *
# cls = df2.columns
# print(cls)
cools = df2.columns+["abc","pqr"]
print(cools)
for i in cools:

    if i not in df2.columns:
        df2 = df2.withColumn(i,lit(None))
df2.show()

In [ ]:
cols = ["EMPLOYEE_ID","JOB_ID"]
for i in df2.columns:
    if i in cols:
        df2 = df2.withColumnRenamed(i,i+"_1")

df2.show()

In [ ]:
from pyspark.sql.functions import *
df3 =df2.select([count(when(col(i).isNull(),i)).alias(i) for i in df2.columns]).show()

In [ ]:
from pyspark.sql.functions import spark_partition_id
df2 = spark.read.csv("employees.csv",header = "true",inferSchema = "true")
df6 = df2.dropDuplicates(["JOB_ID"])
df7 =df6.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count()
# df7.show()

In [ ]:
print(df7.rdd.getNumPartitions())

In [ ]:
df9 = df7.repartition(5)

In [ ]:
print(df9.rdd.getNumPartitions())

In [ ]:
from pyspark.sql.functions import asc
df11 =df9.withColumn("partitionId",spark_partition_id()).orderBy(asc("partitionid")).groupBy("partitionId").count().show()

In [ ]:
lstdf2 = df2.columns
print(lstdf2)

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import DateType
from pyspark.sql.functions import row_number,rank
import datetime
df2.printSchema()
# w =Window.partitionBy("DEPARTMENT_ID").orderBy("SALARY")
# df4 = df2.withColumn("HIRE_DATE",to_date(col("HIRE_DATE").cast(DateType())))
# df4.printSchema()
# .withColumn("date",to_date(current_date(),"dd-MMM-yy")).withColumn("chk",lit(None))

df3 = df2.withColumn("date_diff",datediff(to_date(current_date(),"dd-MMM-yy"),col("HIRE_DATE").cast(DateType())))

df3.printSchema()

# df6 = df4.select([count(when(col(i)),i).alias(i) for i in df4.columns]).show()
# lstdf4 = df4.columns
# print(lstdf4)
# df6 =df4.select([count(when(col(c).isNotNull(),c)).alias(c) for c in df4.columns]).show()


In [ ]:
# allcol = lstdf2+lstdf4

# for i in allcol:
#     if i not in lstdf4:
#         df4 = df4.withColumn(i,lit(None))
#     if i not in lstdf2:
#         df2 = df2.withColumn(i,lit(None))
# df4.show()
# df2.show()

In [ ]:
# df3 = spark.sql("select * from chk a join chk b on a.EMPLOYEE_ID = b.MANAGER_ID").show()

In [ ]:
def addmissingcolumn(df1,df2):
    from pyspark.sql.functions import lit
    allcolumns = df1.columns+df2.columns
    print("allcoumns:",allcolumns)
    for i in allcolumns:
        if i not in df1.columns:
            df1 = df1.withColumn(i,lit(None))
        if i not in df2.columns:
            df2.df2.withColumn(i,lit(None))
    return df1,df2

In [ ]:
addmissingcolumn(df2,df4)
df2.show()
df4.show()

In [ ]:
df6.write.format('delta').mode('overwrite').save("chk")

In [ ]:

{
    "conf":{
        "spark.sql.extensions":"io.delta.sql.DeltaSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.delta.catalog.DeltaCatalog"
    }
}